In [ ]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv() 
import os 
os.environ["LOGURU_LEVEL"] = "INFO"

from oxeo.water.models.tile_utils import predict_tile
from oxeo.core.models.tile import TilePath, tile_from_id, Tile
from oxeo.water.models.segmentation import Segmentation2DPredictor

In [ ]:
# Load hand label json files
# Import the Google Cloud client library and JSON library
from google.cloud import storage
import json

# Instantiate a Google Cloud Storage client and specify required bucket and file
storage_client = storage.Client()
bucket = storage_client.get_bucket('oxeo-handlabelling')
blob = bucket.blob('iris/zimmoz-landsat-5_zimmoz.json')

# Download the contents of the blob as a string and then parse it using json.loads() method
data = json.loads(blob.download_as_string(client=None))

In [ ]:
from typing import List
def get_tile_paths(data: dict) -> List[TilePath]:
    tile_paths = []
    for _, tile_data in data.items():
        tile = tile_from_item(tile_data)
        tile_paths.append(TilePath(tile=tile, constellation=tile_data["spacecraft_id"], root="gs://oxeo-water/prod2"))
    return tile_paths

In [ ]:
def tile_from_item(item: dict) -> Tile:
    return tile_from_id(item["tile"])

In [ ]:
tile_paths = get_tile_paths(data)

In [ ]:


import gcsfs
fs = gcsfs.GCSFileSystem()
predictor = Segmentation2DPredictor(ckpt_path="gs://oxeo-models/semseg/last.ckpt", fs=fs)
predict_tile(tile_paths[0], "cnn", predictor, 
            revisit_chunk_size=4, 
            start_date="1993-03-19", end_date="2000-03-19",
             fs=fs)